In [1]:
%load_ext autoreload
%autoreload 2
import torch
from PIL import Image
from torchvision import transforms
import os
import numpy as np
import pickle


# THE CELL BELLOW CONTAINS THE FILENAMES TO CHANGE
This script follows this pytorch tutorial: https://pytorch.org/hub/pytorch_vision_resnet/

In [2]:
LABELED_DIR = '../data/CrisisMMD_v1.0/data_image/california_wildfires/'
UNLABELED_DIR = '../data/retrieved_data/images/calfire_images_complete/'
NPY_OUTPUT_DIR = '../data/extracted_features/resnet/calfire/'

In [3]:
# load the model

model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)
model.eval()
model.fc = torch.nn.Identity() # replace the last layer with a pass thru

Using cache found in /home/jack/.cache/torch/hub/pytorch_vision_master


In [4]:
# define image preprocessing
preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

# First get labeled data

In [5]:
date_dirs =os.listdir(LABELED_DIR)

count = 0
features_vectors = {}
image_sizes = {}

for date_dir in date_dirs:
    if date_dir == '.DS_Store':
            continue
    filepath = LABELED_DIR + date_dir
    for filename in os.listdir(filepath):
        if filename == '.DS_Store':
            continue
        input_image = Image.open(filepath +'/' +filename)
        image_sizes[filename] = input_image.size
        
        # remove alpha channel if transperency 
        # https://stackoverflow.com/questions/9166400/convert-rgba-png-to-rgb-with-pil
        if input_image.mode == 'RGBA':
            input_image = Image.fromarray(np.array(input_image)[:,:,:3], 'RGB')
            
        # handle palleted images NOTE THIS WILL STILL RAISE WARNINGS but we are just removing the alpha channel
        # https://stackoverflow.com/questions/52307290/what-is-the-difference-between-images-in-p-and-l-mode-in-pil
        # also handle black and white images 'L'
        elif input_image.mode == 'P' or input_image.mode == 'L':
            input_image = input_image.convert('RGB')
        
        # handle palleted images NOTE THIS WILL STILL RAISE WARNINGS but we are just removing the alpha channel
        # https://stackoverflow.com/questions/52307290/what-is-the-difference-between-images-in-p-and-l-mode-in-pil
        elif input_image.mode == 'P':
            input_image = input_image.convert('RGB')
        
        input_tensor = preprocess(input_image)
            
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
        # move the input and model to GPU for speed if available
        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            model.to('cuda')
        
        with torch.no_grad():
            features_vectors[filename] = model(input_batch)[0].cpu().numpy()
        
        count += 1
        if count % 200 == 199:
            print(count)

199
399
599
799
999
1199
1399


In [6]:
# save as a dictionary {filename : np.array}

with open(NPY_OUTPUT_DIR + 'labeled.npz', 'wb+') as fout:
    np.savez_compressed(fout, **features_vectors)
    

In [7]:
with open(NPY_OUTPUT_DIR + 'image_sizes_labeled.p', 'wb') as fout:
    pickle.dump(image_sizes, fout)

In [8]:
# load example
#with open(NPY_OUTPUT_DIR + 'image_sizes.p', 'rb') as fin:
    #image_sizes = pickle.load(fin)
    
    
    

# Next get unlabeled data

In [9]:
count = 0
features_vectors = {}
image_sizes = {}

for filename in os.listdir(UNLABELED_DIR):
    if filename == '.DS_Store':
        continue
    input_image = Image.open(UNLABELED_DIR + filename)
    image_sizes[filename] = input_image.size
    
    # remove alpha channel if transperency 
    # https://stackoverflow.com/questions/9166400/convert-rgba-png-to-rgb-with-pil
    if input_image.mode == 'RGBA':
        input_image = Image.fromarray(np.array(input_image)[:,:,:3], 'RGB') 
        
    # handle palleted images NOTE THIS WILL STILL RAISE WARNINGS but we are just removing the alpha channel
    # https://stackoverflow.com/questions/52307290/what-is-the-difference-between-images-in-p-and-l-mode-in-pil
    # also handle black and white images 'L'
    elif input_image.mode == 'P' or input_image.mode == 'L':
       input_image = input_image.convert('RGB')
    
    input_tensor = preprocess(input_image)
        
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')
    
    with torch.no_grad():
        features_vectors[filename] = model(input_batch)[0].cpu().numpy()
    
    count += 1
    if count % 200 == 199:
        print(count)
        

/home/jack/anaconda3/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


199
399
599
799
999
1199
1399
1599
1799
1999
2199
2399
2599
2799
2999
3199


In [10]:
# save as a dictionary {filename : np.array}

with open(NPY_OUTPUT_DIR + 'unlabeled.npz', 'wb+') as fout:
    np.savez_compressed(fout, **features_vectors)
    

In [11]:
with open(NPY_OUTPUT_DIR + 'image_sizes_unlabeled.p', 'wb') as fout:
    pickle.dump(image_sizes, fout)